# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Практикум 7.2. Управление браузером с Selenium: скроллинг

Импортируем функцию `sleep` и необходимые инструменты для работы с Selenium:m

In [1]:
from time import sleep
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

Запустим новую сессию работы с Selenium – откроем окно браузера на весь экран:

In [2]:
br = wd.Chrome()
br.maximize_window()

Перейдем на сайт онлайн-магазина Гжельского завода:

In [3]:
url = "https://farfor-gzhel.ru/internetmagazin/922/filter/color_list-is-deaf-cobalt/apply/"

### Задача 1

Найдите, используя XPATH, все фрагменты кода HTML с названием товара и ссылкой на его страницу. Пример фрагмента для определенности:

    <a class="section-item-name intec-cl-text-hover" href="/internetmagazin/chaynye-pary/31847/" target="_blank" data-role="offer.link" data-id="31847">Чайная пара Граненая в глухом кобальте объем 330 мл.</a>
    
Результат – список `htmls` со строками, содержащими HTML-код.

In [4]:
br.get(url)

In [5]:
divs = br.find_elements(By.XPATH, "//div[@class='catalog-section-item-name']")
htmls = [d.get_attribute("innerHTML") for d in divs]
print(htmls[0])

<a class="section-item-name intec-cl-text-hover" href="/internetmagazin/chaynye-pary/31847/" target="_blank" data-role="offer.link" data-id="31847">Чайная пара Граненая в глухом кобальте объем 330 мл.</a>


### Задача 2

Найдите, используя поиск по названию атрибута `class`, кнопку «Показать еще» (это будет объект типа `WebElement`). Извлеките ее положение на странице и сохраните ее вертикальную координату в переменную `y`.

**Подсказка:** положение на странице – атрибут `.location`.

In [6]:
more = br.find_element(By.CLASS_NAME, "catalog-section-more")
print(more.location)

# x, y – координаты кнопки на страницы в пикселях,
# у каждого свои – зависит от размера экрана ноутбука
y = more.location["y"]

{'x': 310, 'y': 5431}


Библиотека Selenium умеет скроллить страницы, точнее, активировать запуск кода на JavaScript, который отвечает за скроллинг. В общем виде строка с кодом для скроллинга выглядит так (`Y` – на сколько пикселей нужно проскроллить):

    br.execute_script("window.scrollTo(0, Y);") 

Если нужно проскроллить до конца страницы, то тогда вместо `Y` нужно вписать значение, которое извлекается из тела документа HTML:

    document.body.scrollHeight

### Задача 3

Проскролльте страницу до конца. Попробуйте найти кнопку «Показать еще». Что замечаете?

In [7]:
br.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [8]:
# после скроллинга кнопка находится на границах страницы, 
# не исключено, что на другом компьютере после скроллинга 
# кнопка останется вне зоны видимости, особенно, если мы добавим еще 
# результаты поиска – тогда она не будет обнаружена через find_element()

### Задача 4

Предложите более удачный вариант скроллинга и напишите код, который подставляет необходимое число пикселей для скроллинга вниз в f-строку. Пример – в `{}` подставить нужное число:

    script = f"window.scrollTo(0, {});"
    br.execute_script(script)

In [9]:
# скроллим не до конца страницы, а до точки чуть выше кнопки
# координата y минус 100 пикселей

script = f"window.scrollTo(0, {y - 100});"
br.execute_script(script)

### Задача 5

Допишите цикл `while` ниже, который будет скроллить страницу и извлекать результаты до тех пор, пока они не закончатся, то есть пока на странице будет кнопка «Показать еще». 

Примечание: чтобы в случае отсутствия кнопки код не выдавал ошибку `NoSuchElementException`, а возвращал пустой результат, в коде ниже используется не метод `.find_element()`, а `.find_elements()`. Этот метод ищет все совпадения, не только первое, поэтому в случае отсутствия результатов вместо ошибки возвращается пустой список.

**NB.** Не забудьте добавить небольшую задержку в конце каждой итерации цикла.

In [10]:
br = wd.Chrome()
br.maximize_window()
br.get(url)

all_results = []

# while True = бесконечный вариант цикла for,
# код будет исполняться до тех пор, пока не столкнется с break
# или ошибкой

# если кнопка «Показать еще» найдена (есть куда скроллить далее),
# список more_list не пустой (его длина больше 0),
# мы продолжаем скроллить и извлекать результаты,
# если иначе – выходим из цикла через break

while True:
    more_list = br.find_elements(By.CLASS_NAME, 
                                "catalog-section-more")
    
    if len(more_list) > 0:
        
        # выбираем единственный элемент списка – кнопку
        # скроллим страницу до нее
        
        more = more_list[0]
        y = more.location["y"]
        script = f"window.scrollTo(0, {y - 100});"
        br.execute_script(script)
        
        # забираем фрагменты HTML с информацией о товарах
        # добавляем их в список all_results
        
        divs = br.find_elements(By.XPATH, 
                                "//div[@class='catalog-section-item-name']")
        htmls = [d.get_attribute("innerHTML") for d in divs]
        all_results.extend(htmls)
        
        # как собрали – кликаем на кнопку и немного ждем
        # далее возвращаемся к началу цикла и скроллим дальше
        more.click()
        sleep(2)
    else:
        break

**Дополнение 1.** Полученные результаты нужно обработать. При скроллинге почти всегда возникает дублирование информации – при перелистывании часть результатов оказывается «на стыке» вида страницы до скроллинга и после, а значит, собирается дважды. Проверим длину списка с результатами:

In [11]:
print(len(all_results))

180


Так как порядок элементов здесь не важен (мы просто сгрузили все товары в данном разделе, без сортировки), можем превратить список в множество, чтобы избавиться от дубликатов, а затем – снова в список, потому что с ним удобнее работать, чем с множеством:

In [12]:
uniq_results = list(set(all_results))
print(len(uniq_results))

72


Один элемент `uniq_results` – это просто текст с кодом HTML:

In [13]:
print(uniq_results[0])


                <a class="section-item-name intec-cl-text-hover" href="/internetmagazin/fruktovnitsy/12608/" target="_blank" data-role="offer.link" data-id="12608">Фруктовница Идиллия кобальт краски зол объем 550 мл.</a>            


Можем импортировать `BeautifulSoup` и извлечь название и ссылку:

In [14]:
from bs4 import BeautifulSoup

In [15]:
# превращаем все строки с HTML в объекты BeautifulSoup

soups = [BeautifulSoup(u) for u in uniq_results]

In [16]:
pairs = []

for s in soups:
    a = s.find("a")
    title = a.text
    link = a.get("href")
    full_link = "https://farfor-gzhel.ru" + link
    pairs.append([title, full_link])

In [17]:
# пример результата
print(pairs[0])

['Фруктовница Идиллия кобальт краски зол объем 550 мл.', 'https://farfor-gzhel.ru/internetmagazin/fruktovnitsy/12608/']


**Дополнение 2.** Превратим список пар выше в датафрейм:

In [18]:
import pandas as pd

In [19]:
df = pd.DataFrame(pairs)

In [20]:
df.columns = ["title", "url"]

In [21]:
# первые 5 строк
df.head()

,title,url
0,Фруктовница Идиллия кобальт краски зол объем 5...,https://farfor-gzhel.ru/internetmagazin/frukto...
1,"Кувшин Полдень (кобальт, краски, золото) объем...",https://farfor-gzhel.ru/internetmagazin/kuvshi...
2,Чайник Пышка кобальт краски золото объем 450 мл.,https://farfor-gzhel.ru/internetmagazin/chayni...
3,"Конфетница идиллия (кобальт, краски, золото)",https://farfor-gzhel.ru/internetmagazin/konfet...
4,Чайная пара Орхидея Ночной сад 220 мл.,https://farfor-gzhel.ru/internetmagazin/chayny...


У некоторых товаров указан объем (у кувшинов, чашек и подобных изделий), можем через метод `extract()` подмодуля `str` в `pandas` его извлечь. И тут пригодятся регулярные выражения и группы:

In [22]:
df["title"].str.extract("объем (\d+) мл")

,0
0,550
1,1768
2,450
3,NaN
4,NaN
...,...
67,360
68,187
69,170
70,NaN


Добавим полученный столбец в датафрейм:

In [23]:
df["volume"] = df["title"].str.extract("объем (\d+) мл")
df.head(10)

,title,url,volume
0,Фруктовница Идиллия кобальт краски зол объем 5...,https://farfor-gzhel.ru/internetmagazin/frukto...,550
1,"Кувшин Полдень (кобальт, краски, золото) объем...",https://farfor-gzhel.ru/internetmagazin/kuvshi...,1768
2,Чайник Пышка кобальт краски золото объем 450 мл.,https://farfor-gzhel.ru/internetmagazin/chayni...,450
3,"Конфетница идиллия (кобальт, краски, золото)",https://farfor-gzhel.ru/internetmagazin/konfet...,NaN
4,Чайная пара Орхидея Ночной сад 220 мл.,https://farfor-gzhel.ru/internetmagazin/chayny...,NaN
5,"Сервиз чайно-кофейный Чародейка ""Зимний"" в глу...",https://farfor-gzhel.ru/internetmagazin/serviz...,NaN
6,"Чайник Юность (кобальт, краски, золото) объем ...",https://farfor-gzhel.ru/internetmagazin/chayni...,1050
7,Набор столовый Дубок: глухой кобальт и золото,https://farfor-gzhel.ru/internetmagazin/serviz...,NaN
8,Креманка вазочка для варенья сластена кобальт ...,https://farfor-gzhel.ru/internetmagazin/kreman...,NaN
9,Салфетница Идиллия глухой кобальт,https://farfor-gzhel.ru/internetmagazin/salfet...,NaN


Отдельная долгоиграющая задача – придумать выражение, которое поможет извлечь тип изделия. С одной стороны, можно брать из `title` все, с первого символа до следующей заглавной буквы (`Чайная пара Орхидея` – `Чайная пара`, `Сервиз чайно-кофейный Чародейка` – `Сервиз чайно-кофейный`). С другой стороны, есть примеры, где название идет не с большой буквы (`Креманка вазочка для варенья сластена` и `Конфетница идиллия`). Не исключено, что проще написать отдельный парсер, который по ссылке на товар будет выгружать и добавлять более подробную информацию  :) 